## 배치마다 저장되도록 수정/ 생성된 데이터 추후 가공해야되지만 일단 Go!

### 질의 생성 -> 가공 -> 질의-문서로 pos, neg 레이블 만들기

- 무엇->뭐
- 시작할 때 나오는 숫자. 지우기

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
import os

In [2]:
# 모델과 토크나이저 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "NCSOFT/Llama-VARCO-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # float16 사용으로 메모리 절약
    device_map="auto"
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# Special tokens 추가 및 임베딩 업데이트
special_tokens = ["<|eot_id|>"]
tokenizer.add_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(128256, 4096)

In [4]:
# 문서 데이터 로드
data_path = '../data/documents_copy.jsonl'
if not os.path.exists(data_path):
    raise FileNotFoundError(f"파일 {data_path}이(가) 존재하지 않습니다. 경로를 확인해 주세요.")

data = []
with open(data_path, 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line))

In [5]:
# 이미 처리된 데이터 확인
output_path = '../data/triples_data.jsonl'
processed_doc_ids = set()

if os.path.exists(output_path):
    with open(output_path, 'r', encoding='utf-8') as output_file:
        for line in output_file:
            entry = json.loads(line)
            processed_doc_ids.add(entry['docid'])


In [6]:
# 질문 생성 함수
def generate_questions(content, model, tokenizer, device, num_questions=3):
    prompt = f"다음 내용을 바탕으로 이해를 돕기 위한 {num_questions}개의 질문을 반말로 생성해줘:\n내용: {content[:500]}\n질문:"  # 입력 길이 제한
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

    eos_token_id = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>"),
    ]

    with torch.no_grad():  # 메모리 사용 줄이기
        outputs = model.generate(
            **inputs,
            eos_token_id=eos_token_id,
            max_length=512,  # max_length 감소
            num_return_sequences=1,
            do_sample=True,
            temperature=0.6,
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    questions = generated_text.split('질문:')[-1].strip().split('\n')[:num_questions]
    formatted_questions = {f'question{i+1}': question.strip() for i, question in enumerate(questions)}
    return formatted_questions


In [7]:
# 트리플 데이터 준비 및 저장
batch_size = 2
new_triples_data = []

for idx, entry in enumerate(data):
    doc_id = entry['docid']

    # 이미 처리된 docid는 건너뛰기
    if doc_id in processed_doc_ids:
        continue

    content = entry['content']
    questions = generate_questions(content, model, tokenizer, device)
    new_triples_data.append({
        'docid': doc_id,
        'content': content,
        'questions': questions,
    })
    torch.cuda.empty_cache()  # 캐시 정리로 메모리 확보

    # 100개마다 새로 생성된 데이터만 저장
    if (len(new_triples_data)) % batch_size == 0 or (idx + 1) == len(data):
        with open(output_path, 'a', encoding='utf-8') as output_file:  # 'a' 모드로 추가 저장
            for entry in new_triples_data:
                json.dump(entry, output_file, ensure_ascii=False)
                output_file.write('\n')
        new_triples_data = []  # 저장 후 리스트 초기화
        print(f"{idx + 1}개의 새 데이터가 {output_path}에 추가 저장되었습니다.")

print(f"모든 새로 생성된 트리플 데이터가 {output_path}에 저장되었습니다.")


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


2개의 새 데이터가 ../data/triples_data.jsonl에 추가 저장되었습니다.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


4개의 새 데이터가 ../data/triples_data.jsonl에 추가 저장되었습니다.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


6개의 새 데이터가 ../data/triples_data.jsonl에 추가 저장되었습니다.
모든 새로 생성된 트리플 데이터가 ../data/triples_data.jsonl에 저장되었습니다.
